In [1]:
from gol_apis import RosterActivitiesService
from gol_apis import RosterTagService
from gol_apis import CheckScheduleService
from gol_apis import CrewMemberInfoService
import api_tools
from api_tools.setup import setup_crew_info_dataframe
from api_tools.setup import setup_checks_dataframe
from api_tools.setup import setup_tag_dataframe
from api_tools.setup import setup_activities_dataframe

from gol_logger import Logger
import pandas as pd
import multiprocessing as mp
import requests, time
from datetime import datetime as dt
from datetime import timedelta as td
from utils import funcs
from utils.progress import step
from utils.progress import BAR
from itertools import chain

from modules.generate_data import get_data_neolude
from modules.generate_data import get_rosters
from modules.generate_data import get_crew_info
from modules.generate_data import get_checks

import sys
from config.variables import user

In [2]:
training_code = 'EMG'

In [3]:
# Dados Crew Info
crew = pd.read_csv('data/crew/crew_info.csv', sep=';', encoding='utf-8-sig')
crew.cif = crew.cif.astype(int).astype(str).str.zfill(8)
crew = crew[crew['funcao'].isin(['CMT', 'COP'])]
crew = crew[['cif', 'guerra', 'funcao', 'cargo', 'base']]
crew

,cif,guerra,funcao,cargo,base
0,00000020,BORIO,CMT,IBX,CGH
1,00000021,GRABLER,CMT,IBX,GRU
2,00000022,DANTAS,CMT,IBX,CGH
3,00000024,BORBA,CMT,NaN,GRU
4,00000026,FERNANDO FARIAS,CMT,IBX,CGH
...,...,...,...,...,...
5479,99999913,PIAZZA,CMT,IB,GRU
5480,99999916,GALENO CABRAL,CMT,IB,GRU
5481,99999917,EDWARD,CMT,IB,GRU
5482,99999918,CESIDIO,CMT,IB,GRU


In [4]:
# Header para filtrar dados do Neolude
header_neolude = ['RegistrationNumber',
                  'UserName',
                  'BusinessUnitName',
                  'Post',
                  'CPF',
                  'Email',
                  'CourseName',
                  'CourseUserStatus',
                  'GradeFormatted',
                  'LastAccessDate',
                  'CreatedDate',
                  'CompletionDate']

In [5]:
# Dados Neolude
training = pd.read_csv(f'data/neolude/{training_code}.csv', sep=';', encoding='utf-8-sig')
training = training[header_neolude]
training['tipo'] = pd.to_numeric(training['RegistrationNumber'], errors='coerce').notnull()
training = training[training['tipo']]
training['RegistrationNumber'] = training['RegistrationNumber'].fillna(0)
training['RegistrationNumber'] = training['RegistrationNumber'].astype(int).astype(str).str.zfill(8)
training['LastAccessDate'] = pd.to_datetime(training['LastAccessDate'])
training['CreatedDate'] = pd.to_datetime(training['CreatedDate'])
training['CompletionDate'] = pd.to_datetime(training['CompletionDate'])
training.columns = ['cif', 'name', 'base', 'funcao', 'cpf', 'email', 'course_name','status', 'nota', 'ultimo_acesso', 'criacao_acesso', 'data_conclusao', 'tipo']
training = training[['cif', 'course_name','status', 'nota', 'data_conclusao']]
training = training.sort_values(by=['cif','data_conclusao'])
training = training.drop_duplicates(subset = ['cif'], keep = 'last')
training

,cif,course_name,status,nota,data_conclusao
1536,00000021,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"10,00",2020-12-05 09:51:53
1431,00000022,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"8,00",2020-09-08 15:44:08
1368,00000024,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"8,00",2020-08-04 15:28:16
1136,00000026,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"10,00",2020-12-18 00:31:26
1108,00000027,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"9,00",2020-08-06 11:35:56
...,...,...,...,...,...
949,00044789,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"9,00",2020-12-14 15:37:03
1549,00044791,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"8,00",2020-12-05 14:41:36
1064,00044794,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"9,00",2020-12-06 17:51:30
1332,00044795,Tripulação Técnica - Emergências Gerais e Trei...,Em andamento,"0,00",NaT


In [6]:
# Dados Neolude
training_p = pd.read_csv(f'data/neolude/{training_code}_presencial.csv', sep=';', encoding='utf-8-sig')
training_p = training_p[header_neolude]
training_p['tipo'] = pd.to_numeric(training_p['RegistrationNumber'], errors='coerce').notnull()
training_p = training_p[training_p['tipo']]
training_p['RegistrationNumber'] = training_p['RegistrationNumber'].fillna(0)
training_p['RegistrationNumber'] = training_p['RegistrationNumber'].astype(int).astype(str).str.zfill(8)
training_p['LastAccessDate'] = pd.to_datetime(training_p['LastAccessDate'])
training_p['CreatedDate'] = pd.to_datetime(training_p['CreatedDate'])
training_p['CompletionDate'] = pd.to_datetime(training_p['CompletionDate'])
training_p.columns = ['cif', 'name', 'base', 'funcao', 'cpf', 'email', 'course_name_p','status_p', 'nota_p', 'ultimo_acesso', 'criacao_acesso', 'data_conclusao_p', 'tipo']
training_p = training_p[['cif', 'course_name_p','status_p', 'nota_p', 'data_conclusao_p']]
training_p = training_p.sort_values(by=['cif','data_conclusao_p'])
training_p = training_p.drop_duplicates(subset = ['cif'], keep = 'last')
training_p

,cif,course_name_p,status_p,nota_p,data_conclusao_p
1025,00000020,Tripulação Técnica - Emergências Gerais Práticas,Concluído,"0,00",2020-09-08 11:25:22
1006,00000021,Tripulação Técnica - Emergências Gerais Práticas,Concluído,"0,00",2021-01-11 17:00:00
939,00000022,Tripulação Técnica - Emergências Gerais Práticas,Concluído,"0,00",2020-08-20 08:58:43
893,00000024,Tripulação Técnica - Emergências Gerais Práticas,Concluído,"0,00",2020-08-28 20:05:21
722,00000027,Tripulação Técnica - Emergências Gerais Práticas,Concluído,"0,00",2020-08-19 08:37:52
...,...,...,...,...,...
392,00044690,Tripulação Técnica - Emergências Gerais Práticas,Concluído,"0,00",2021-01-04 17:45:00
375,00044698,Tripulação Técnica - Emergências Gerais Práticas,Concluído,"0,00",2021-01-04 17:45:00
847,00044768,Tripulação Técnica - Emergências Gerais Práticas,Concluído,"0,00",2021-01-04 17:45:00
748,00044771,Tripulação Técnica - Emergências Gerais Práticas,Concluído,"0,00",2021-01-04 17:45:00


In [7]:
# Dados de Rosters
rosters = funcs.generate_data('data/rosters/')
rosters.cif = rosters.cif.astype(str).str.zfill(8)
rosters

importing: 2020-01...
importing: 2020-02...
importing: 2020-03...
importing: 2020-04...
importing: 2020-05...
importing: 2020-06...
importing: 2020-07...
importing: 2020-08...
importing: 2020-09...
importing: 2020-10...
importing: 2020-11...
importing: 2020-12...
importing: 2021-01...


,cif,code,act_type,airline,fleet,tail,leg_type,fr,to,dep,arr,pairingId
0,00010103,SOBREAVIS,A,NaN,NaN,NaN,NaN,CGH,CGH,2020-01-01 06:00:00,2020-01-01 06:05:00,6173246.0
1,00006285,FP,A,NaN,NaN,NaN,NaN,CGH,CGH,2020-01-01 06:00:00,2020-01-02 06:00:00,NaN
2,00022301,SOBREAVIS,A,NaN,NaN,NaN,NaN,CGH,CGH,2020-01-01 06:00:00,2020-01-01 12:00:00,6099354.0
3,00036105,FANI,A,NaN,NaN,NaN,NaN,BSB,BSB,2020-01-01 06:00:00,2020-01-02 06:00:00,NaN
4,00013384,SOBREAVIS,A,NaN,NaN,NaN,NaN,POA,POA,2020-01-01 06:00:00,2020-01-01 14:15:00,6173917.0
...,...,...,...,...,...,...,...,...,...,...,...,...
202069,00023217,G3 1703,L,G3,73M,NaN,M,REC,CGH,2021-02-01 05:50:00,2021-02-01 09:10:00,6720278.0
202070,00010475,FANI,A,NaN,NaN,NaN,NaN,GRU,GRU,2021-02-01 05:50:00,2021-02-02 05:50:00,NaN
202071,00024261,G3 1703,L,G3,73M,NaN,M,REC,CGH,2021-02-01 05:50:00,2021-02-01 09:10:00,6720235.0
202072,00024124,FP,A,NaN,NaN,NaN,NaN,GRU,GRU,2021-02-01 05:55:00,2021-02-02 05:55:00,NaN


In [8]:
inss = rosters.copy()
inss['dep'] = pd.to_datetime(inss['dep'], format='%Y-%m-%d')
inss['dep'] = inss['dep'].dt.floor('D')
inss = inss[(inss['code'] == 'INSS') & (inss['dep'] == pd.to_datetime(dt.date(dt.now())))]
inss = inss[['cif', 'code']]
inss.columns = ['cif', 'inss']
inss

,cif,inss
65631,00001962,INSS
65632,00002329,INSS
65634,00002976,INSS
65643,00010705,INSS
65648,00004196,INSS
...,...,...
66957,00005916,INSS
66959,00006334,INSS
66976,00041650,INSS
66978,00042249,INSS


In [9]:
lnr = rosters.copy()
lnr['dep'] = pd.to_datetime(lnr['dep'], format='%Y-%m-%d')
lnr['dep'] = lnr['dep'].dt.floor('D')
lnr = lnr[(lnr['code'] == 'LIC-SREM') & (lnr['dep'] == pd.to_datetime(dt.date(dt.now())))]
lnr = lnr[['cif', 'code']]
lnr.columns = ['cif', 'lnr']
lnr

,cif,lnr
65635,00023872,LIC-SREM
65636,00023903,LIC-SREM
65638,00024141,LIC-SREM
65649,00004453,LIC-SREM
65651,00004615,LIC-SREM
...,...,...
66972,00046284,LIC-SREM
66973,00046328,LIC-SREM
66974,00046330,LIC-SREM
66975,00046333,LIC-SREM


In [10]:
rosters_o = rosters[rosters['code'].isin(funcs.training_filter('EMG'))]
rosters_o = rosters_o.sort_values(by=['cif','dep'])
rosters_o = rosters_o[['cif', 'code', 'dep']]
rosters_o.columns = ['cif', 'code_on', 'date_on']
rosters_o = rosters_o.drop_duplicates(subset = ['cif'], keep = 'last')
rosters_o

,cif,code_on,date_on
67752,00000021,C-EMG-ON,2020-12-11 19:00:00
12701,00000022,C-EMG-ON,2020-08-03 16:30:00
18435,00000024,C-EMG-ON,2020-08-04 16:30:00
106422,00000026,C-EMG-ON,2020-12-17 20:20:00
34932,00000027,C-EMG-ON,2020-08-07 16:30:00
...,...,...,...
105469,00044789,C-EMG-ON,2020-12-17 16:30:00
67998,00044791,C-EMG-ON,2020-12-11 20:20:00
67997,00044794,C-EMG-ON,2020-12-11 20:20:00
109586,00044795,C-EMG-ON,2021-01-17 16:30:00


In [11]:
rosters_p = rosters[rosters['code'].isin(funcs.training_filter('EMG_presencial'))]
rosters_p = rosters_p.sort_values(by=['cif','dep'])
rosters_p = rosters_p[['cif', 'code', 'dep']]
rosters_p.columns = ['cif', 'code_p', 'date_p']
rosters_p = rosters_p.drop_duplicates(subset = ['cif'], keep = 'last')
rosters_p

,cif,code_p,date_p
33550,00000020,C-ENS-EMG,2020-03-05 16:30:00
70798,00000021,C-ENS-EMG,2021-01-11 17:45:00
104746,00000022,C-ENS-EMG,2020-08-20 12:30:00
147999,00000024,C-ENS-EMG,2020-08-28 11:30:00
98850,00000027,C-ENS-EMG,2020-08-19 11:30:00
...,...,...,...
39143,00045989,C-ENS-EMG,2020-03-06 11:15:00
39196,00045992,C-ENS-EMG,2020-03-06 11:15:00
39166,00045994,C-ENS-EMG,2020-03-06 11:15:00
39111,00045997,C-ENS-EMG,2020-03-06 11:15:00


In [12]:
# Dados de Checks
checks = pd.read_csv('data/checks/check_schedule.csv', sep=';', encoding='utf-8-sig')
checks.cif = checks.cif.astype(str).str.zfill(8)
checks = checks[checks['checkTypeCode'] == training_code]
checks = checks[['cif', 'checkTypeCode', 'expiryDt']]
checks

,cif,checkTypeCode,expiryDt
72,00010655,EMG,2019-12-31
371,00046230,EMG,2022-09-30
374,00023615,EMG,2021-03-30
377,00023617,EMG,2021-03-31
381,00023620,EMG,2021-10-31
...,...,...,...
55316,00008599,EMG,2021-04-30
55321,00006293,EMG,2022-10-31
55328,00032174,EMG,2020-12-29
55329,00023879,EMG,2022-10-31


In [13]:
# Merge dos dados
data_all = pd.merge(crew, checks[['cif', 'checkTypeCode', 'expiryDt']], on='cif', how='left')
data_all = data_all.merge(training, on='cif', how='left')
data_all = data_all.merge(training_p, on='cif', how='left')
data_all = data_all.merge(rosters_o, on='cif', how='left')
data_all = data_all.merge(rosters_p, on='cif', how='left')
data_all = data_all.merge(inss, on='cif', how='left')
data_all = data_all.merge(lnr, on='cif', how='left')
data_all

,cif,guerra,funcao,cargo,base,checkTypeCode,expiryDt,course_name,status,nota,...,course_name_p,status_p,nota_p,data_conclusao_p,code_on,date_on,code_p,date_p,inss,lnr
0,00000020,BORIO,CMT,IBX,CGH,EMG,2021-09-30,NaN,NaN,NaN,...,Tripulação Técnica - Emergências Gerais Práticas,Concluído,"0,00",2020-09-08 11:25:22,NaN,NaN,C-ENS-EMG,2020-03-05 16:30:00,NaN,NaN
1,00000021,GRABLER,CMT,IBX,GRU,EMG,2021-01-28,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"10,00",...,Tripulação Técnica - Emergências Gerais Práticas,Concluído,"0,00",2021-01-11 17:00:00,C-EMG-ON,2020-12-11 19:00:00,C-ENS-EMG,2021-01-11 17:45:00,NaN,NaN
2,00000022,DANTAS,CMT,IBX,CGH,EMG,2021-08-31,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"8,00",...,Tripulação Técnica - Emergências Gerais Práticas,Concluído,"0,00",2020-08-20 08:58:43,C-EMG-ON,2020-08-03 16:30:00,C-ENS-EMG,2020-08-20 12:30:00,NaN,NaN
3,00000024,BORBA,CMT,NaN,GRU,EMG,2021-08-31,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"8,00",...,Tripulação Técnica - Emergências Gerais Práticas,Concluído,"0,00",2020-08-28 20:05:21,C-EMG-ON,2020-08-04 16:30:00,C-ENS-EMG,2020-08-28 11:30:00,NaN,NaN
4,00000026,FERNANDO FARIAS,CMT,IBX,CGH,EMG,2021-01-28,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"10,00",...,NaN,NaN,NaN,NaT,C-EMG-ON,2020-12-17 20:20:00,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2029,99999913,PIAZZA,CMT,IB,GRU,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2030,99999916,GALENO CABRAL,CMT,IB,GRU,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2031,99999917,EDWARD,CMT,IB,GRU,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2032,99999918,CESIDIO,CMT,IB,GRU,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Formatação do dataFrame de Dados
validation = data_all.copy()

# Ajuste de data do vencimento
validation['expiryDt'] = pd.to_datetime(validation['expiryDt'], format='%Y-%m-%d')
validation['expiryDt'] = validation['expiryDt'].dt.floor('D')

# Ajuste de data do treinamento online
validation['date_on'] = validation['date_on'].fillna(pd.to_datetime(dt(2035,12,31)))
validation['date_on'] = pd.to_datetime(validation['date_on'], format='%Y-%m-%d')
validation['date_on'] = validation['date_on'].dt.floor('D')

# Ajuste de data do treinamento presencial
validation['date_p'] = validation['date_p'].fillna(pd.to_datetime(dt(2035,12,31)))
validation['date_p'] = pd.to_datetime(validation['date_p'], format='%Y-%m-%d')
validation['date_p'] = validation['date_p'].dt.floor('D')


# Data da conclusao ON LINE
validation['data_conclusao'] = validation['data_conclusao'].fillna(pd.to_datetime(dt(2035,12,31)))
validation['data_conclusao'] = pd.to_datetime(validation['data_conclusao'], format='%Y-%m-%d')
validation['data_conclusao'] = validation['data_conclusao'].dt.floor('D')

# Data da conclusao Presencial
validation['data_conclusao_p'] = validation['data_conclusao_p'].fillna(pd.to_datetime(dt(2035,12,31)))
validation['data_conclusao_p'] = pd.to_datetime(validation['data_conclusao_p'], format='%Y-%m-%d')
validation['data_conclusao_p'] = validation['data_conclusao_p'].dt.floor('D')


validation['presencial_depois_online'] = validation['date_p'] > validation['date_on']
validation = validation[validation['expiryDt'] < (pd.to_datetime(dt.date(dt.now())) + pd.offsets.MonthBegin(0))]
validation = validation[validation['date_p'] < (pd.to_datetime(dt.date(dt.now())))]


validation['tempo_de_conclusao (dias)'] = pd.to_datetime(dt.date(dt.now())) - validation['data_conclusao']

validation['atualizar'] = (validation['status'] == 'Concluído') & \
                          (validation['status_p'] == 'Concluído') & \
                          (validation['presencial_depois_online'])

validation

,cif,guerra,funcao,cargo,base,checkTypeCode,expiryDt,course_name,status,nota,...,data_conclusao_p,code_on,date_on,code_p,date_p,inss,lnr,presencial_depois_online,tempo_de_conclusao (dias),atualizar
7,00000031,VENZON,CMT,IBX,CGH,EMG,2021-01-31,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"10,00",...,2021-01-08,C-EMG-ON,2020-12-20,C-ENS-EMG,2021-01-08,NaN,NaN,True,23 days,True
66,00000601,BERGMANN,CMT,NaN,POA,EMG,2021-01-31,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"9,00",...,2035-12-31,C-EMG-ON,2020-12-07,C-ENS-EMG,2020-01-08,NaN,NaN,False,35 days,False
122,00001051,BARRIOS,COP,NaN,GRU,EMG,2021-01-31,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"10,00",...,2021-01-08,C-EMG-ON,2020-12-03,C-ENS-EMG,2021-01-08,NaN,NaN,True,40 days,True
138,00001180,SIDNEI,CMT,IR,GRU,EMG,2021-01-31,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"9,00",...,2021-01-08,C-EMG-ON,2021-01-07,C-ENS-EMG,2021-01-08,NaN,NaN,True,4 days,True
227,00002122,ORTEGAL,CMT,NaN,CGH,EMG,2021-01-31,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"9,00",...,2021-01-08,C-EMG-ON,2020-12-08,C-ENS-EMG,2021-01-08,NaN,NaN,True,41 days,True
313,00002916,BERNARDO,CMT,NaN,CGH,EMG,2021-01-31,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"7,00",...,2021-01-08,C-EMG-ON,2020-12-01,C-ENS-EMG,2021-01-08,NaN,NaN,True,41 days,True
442,00004106,SORZA,CMT,NaN,CGH,EMG,2021-01-31,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"9,00",...,2021-01-08,C-EMG-ON,2020-12-04,C-ENS-EMG,2021-01-08,NaN,NaN,True,25 days,True
533,00005391,JETER,CMT,NaN,GRU,EMG,2021-01-28,Tripulação Técnica - Emergências Gerais e Trei...,Em andamento,"0,00",...,2021-01-04,C-EMG-ON,2020-12-09,C-ENS-EMG,2021-01-04,NaN,NaN,True,-5467 days,False
601,00006333,FREYESLEBEN,CMT,NaN,GIG,EMG,2021-01-31,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"10,00",...,2021-01-08,C-EMG-ON,2020-12-04,C-ENS-EMG,2021-01-08,NaN,NaN,True,38 days,True
660,00008139,FERNANDO JOSE,CMT,NaN,CGH,EMG,2021-01-31,Tripulação Técnica - Emergências Gerais e Trei...,Concluído,"10,00",...,2021-01-08,C-EMG-ON,2020-12-16,C-ENS-EMG,2021-01-08,NaN,NaN,True,26 days,True


In [15]:
validation.to_csv(f'./data/output/{training_code}.csv', index=False, sep=';', encoding='utf-8-sig')